In [7]:
FUENTES_DIR = '../Fuentes'         # carpeta donde se encuentran archivos .py auxiliares
DATOS_DIR   = '../Data_Sets/p3/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pylab as plt
from IPython import display
from sklearn import preprocessing
import grafica as gr
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Model
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Input
from sklearn import metrics
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LeakyReLU
from sklearn.compose import ColumnTransformer

## Ejercicio 9

Utilizando los ejemplos del archivo **AUTOS.csv** genere un modelo utilizando un multiperceptrón para predecir el precio del auto (atributo **price**) y la cantidad de millas por galón en ruta (**MPG-highway**) en función del resto de los atributos. Recuerde completar los valores faltantes, utilizar normalización y dividir el dataset en entrenamiento y validación (80/20).

Realice 20 ejecuciones independientes de cada configuración seleccionada calculando las épocas promedio y al error cuadrático medio (ECM). Analice los resultados y respalde las afirmaciones referidas a los resultados obtenidos. Utilice un máximo de 1000 épocas con lotes de 50 e implemente una parada temprana con paciencia de 15.

Complete la siguiente tabla y realice un análisis de los valores obtenidos:

<table style="border-collapse: collapse; width: 100%; text-align: center;">
  <thead>
    <tr style="background-color:#2f8fbd; color: white;">
      <th>Optimizador</th>
      <th>Función activación</th>
      <th>Épocas promedio</th>
      <th>ECM Promedio</th>
    </tr>
  </thead>
  <tbody>
    <tr style="background-color:#2f8fbd; color: white;">
      <td rowspan="4">SGD</td>
      <td>tanh</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>sigmoid</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>ReLU</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>LeakyReLU</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td rowspan="4">RMSProp</td>
      <td>tanh</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>sigmoid</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>ReLU</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>LeakyReLU</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td rowspan="4">Adam</td>
      <td>tanh</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>sigmoid</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>ReLU</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>LeakyReLU</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
  </tbody>
</table>



Donde:

- **Épocas Promedio** es el número de épocas promedio en el que se detuvo el entrenamiento.
- **ECM Promedio** es el promedio del error de predicción en cada entrenamiento.


## SGD

In [9]:
ALPHA = 0.01
LOSS = 'mse'
EPOCHS = 700
BATCH = 30
REPS = 20
# ------------------------------

# Cargar datos
data = pd.read_csv(DATOS_DIR + 'autos.csv', sep=',')

# Reemplazar ? por NaN
data = data.replace('?', np.nan)

# Convertir columnas numéricas
for col in ['normalized-losses','bore','stroke','horsepower','peak-rpm','price']:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Imputar faltantes
for col in data.select_dtypes(exclude=['object']).columns:
    data[col] = data[col].fillna(data[col].mean())
for col in data.select_dtypes(include=['object']).columns:
    data[col] = data[col].fillna(data[col].mode()[0])

# Features y targets
X = data.drop(columns=['price','highway-mpg'])
T = data[['price','highway-mpg']].astype(float)

# Columnas categóricas y numéricas
cat_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(exclude=['object']).columns

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# Split
X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.2, random_state=42)
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Escalar targets también
target_scaler = StandardScaler()
T_train = target_scaler.fit_transform(T_train)
T_test = target_scaler.transform(T_test)

# ------------------------------
# Función auxiliar para crear el modelo
def build_model(activation):
    I = Input(shape=(X_train.shape[1],), name="entrada")
    if activation == "leakyrelu":
        h1 = Dense(64)(I)
        h1 = LeakyReLU(alpha=0.1)(h1)
        h2 = Dense(32)(h1)
        h2 = LeakyReLU(alpha=0.1)(h2)
    else:
        h1 = Dense(64, activation=activation)(I)
        h2 = Dense(32, activation=activation)(h1)
    out = Dense(2, activation='linear', name="salida")(h2)
    model = Model(inputs=I, outputs=out)
    model.compile(optimizer=SGD(learning_rate=ALPHA), loss=LOSS, metrics=['mse'])
    return model

# ------------------------------
# Entrenamiento repetido
results = {}
activations = ["tanh", "sigmoid", "relu", "leakyrelu"]

for act in activations:
    epochs_list = []
    ecm_list = []
    for r in range(REPS):
        model = build_model(act)
        early = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)
        history = model.fit(
            X_train, T_train,
            epochs=EPOCHS,
            batch_size=BATCH,
            validation_data=(X_test, T_test),
            verbose=0,
            callbacks=[early]
        )
        # épocas hasta detenerse
        epochs_list.append(len(history.history['loss']))
        # ECM en test
        ecm = model.evaluate(X_test, T_test, verbose=0)[0]
        ecm_list.append(ecm)
    results[act] = {
        "epocas_promedio": np.mean(epochs_list),
        "ecm_promedio": np.mean(ecm_list)
    }

# ------------------------------
# Mostrar resultados
print("\nResultados SGD (targets escalados):")
for act, vals in results.items():
    print(f"{act:10s} -> Épocas promedio: {vals['epocas_promedio']:.2f}, ECM promedio: {vals['ecm_promedio']:.4f}")

/home/manuel/Documents/Facultad/DeepLearning/env/lib/python3.13/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(



Resultados SGD (targets escalados):
tanh       -> Épocas promedio: 234.80, ECM promedio: 0.1137
sigmoid    -> Épocas promedio: 700.00, ECM promedio: 0.2144
relu       -> Épocas promedio: 292.90, ECM promedio: 0.1096
leakyrelu  -> Épocas promedio: 277.50, ECM promedio: 0.1112


## RMSprop


In [10]:
ALPHA = 0.01
LOSS = 'mse'
EPOCHS = 700
BATCH = 30
REPS = 20
# ------------------------------

# Cargar datos
data = pd.read_csv(DATOS_DIR + 'autos.csv', sep=',')

# Reemplazar ? por NaN
data = data.replace('?', np.nan)

# Convertir columnas numéricas
for col in ['normalized-losses','bore','stroke','horsepower','peak-rpm','price']:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Imputar faltantes
for col in data.select_dtypes(exclude=['object']).columns:
    data[col] = data[col].fillna(data[col].mean())
for col in data.select_dtypes(include=['object']).columns:
    data[col] = data[col].fillna(data[col].mode()[0])

# Features y targets
X = data.drop(columns=['price','highway-mpg'])
T = data[['price','highway-mpg']].astype(float)

# Columnas categóricas y numéricas
cat_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(exclude=['object']).columns

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# Split
X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.2, random_state=42)
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Escalar targets también
target_scaler = StandardScaler()
T_train = target_scaler.fit_transform(T_train)
T_test = target_scaler.transform(T_test)

# ------------------------------
# Función auxiliar para crear el modelo
def build_model(activation):
    I = Input(shape=(X_train.shape[1],), name="entrada")
    if activation == "leakyrelu":
        h1 = Dense(64)(I)
        h1 = LeakyReLU(alpha=0.1)(h1)
        h2 = Dense(32)(h1)
        h2 = LeakyReLU(alpha=0.1)(h2)
    else:
        h1 = Dense(64, activation=activation)(I)
        h2 = Dense(16, activation=activation)(h1)
    out = Dense(2, activation='linear', name="salida")(h2)
    model = Model(inputs=I, outputs=out)
    model.compile(optimizer=RMSprop(learning_rate=ALPHA), loss=LOSS, metrics=['mse'])
    return model

# ------------------------------
# Entrenamiento repetido
results = {}
activations = ["tanh", "sigmoid", "relu", "leakyrelu"]

for act in activations:
    epochs_list = []
    ecm_list = []
    for r in range(REPS):
        model = build_model(act)
        early = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)
        history = model.fit(
            X_train, T_train,
            epochs=EPOCHS,
            batch_size=BATCH,
            validation_data=(X_test, T_test),
            verbose=1,
            callbacks=[early]
        )
        # épocas hasta detenerse
        epochs_list.append(len(history.history['loss']))
        # ECM en test
        ecm = model.evaluate(X_test, T_test, verbose=1)[0]
        ecm_list.append(ecm)
    results[act] = {
        "epocas_promedio": np.mean(epochs_list),
        "ecm_promedio": np.mean(ecm_list)
    }

# ------------------------------
# Mostrar resultados
print("\nResultados SGD (targets escalados):")
for act, vals in results.items():
    print(f"{act:10s} -> Épocas promedio: {vals['epocas_promedio']:.2f}, ECM promedio: {vals['ecm_promedio']:.4f}")

Epoch 1/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1.1195 - mse: 1.1195 - val_loss: 0.4120 - val_mse: 0.4120
Epoch 2/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1955 - mse: 0.1955 - val_loss: 0.3005 - val_mse: 0.3005
Epoch 3/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1383 - mse: 0.1383 - val_loss: 0.2720 - val_mse: 0.2720
Epoch 4/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1269 - mse: 0.1269 - val_loss: 0.3316 - val_mse: 0.3316
Epoch 5/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2341 - mse: 0.2341 - val_loss: 0.3016 - val_mse: 0.3016
Epoch 6/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1087 - mse: 0.1087 - val_loss: 0.2633 - val_mse: 0.2633
Epoch 7/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1200 - mse: 0.1200 - val_loss: 0.2419 - val_mse: 0.2419
Epoch 8/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1188 - mse: 0.1188 - val_loss: 0.2186 - val_mse: 0.2186
Epoch 9/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0862 - mse: 

/home/manuel/Documents/Facultad/DeepLearning/env/lib/python3.13/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 1.0083 - mse: 1.0083 - val_loss: 0.4465 - val_mse: 0.4465
Epoch 2/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1539 - mse: 0.1539 - val_loss: 0.1925 - val_mse: 0.1925
Epoch 3/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1049 - mse: 0.1049 - val_loss: 0.2649 - val_mse: 0.2649
Epoch 4/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1538 - mse: 0.1538 - val_loss: 0.2112 - val_mse: 0.2112
Epoch 5/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0755 - mse: 0.0755 - val_loss: 0.3172 - val_mse: 0.3172
Epoch 6/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0977 - mse: 0.0977 - val_loss: 0.1453 - val_mse: 0.1453
Epoch 7/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0831 - mse: 0.0831 - val_loss: 0.2861 - val_mse: 0.2861
Epoch 8/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1551 - mse: 0.1551 - val_loss: 0.1817 - val_mse: 0.1817
Epoch 9/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0769 - mse: 0.0769 - val

## Adam

In [11]:
ALPHA = 0.01
LOSS = 'mse'
EPOCHS = 700
BATCH = 30
REPS = 20
# ------------------------------

# Cargar datos
data = pd.read_csv(DATOS_DIR + 'autos.csv', sep=',')

# Reemplazar ? por NaN
data = data.replace('?', np.nan)

# Convertir columnas numéricas
for col in ['normalized-losses','bore','stroke','horsepower','peak-rpm','price']:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Imputar faltantes
for col in data.select_dtypes(exclude=['object']).columns:
    data[col] = data[col].fillna(data[col].mean())
for col in data.select_dtypes(include=['object']).columns:
    data[col] = data[col].fillna(data[col].mode()[0])

# Features y targets
X = data.drop(columns=['price','highway-mpg'])
T = data[['price','highway-mpg']].astype(float)

# Columnas categóricas y numéricas
cat_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(exclude=['object']).columns

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# Split
X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.2, random_state=42)
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Escalar targets también
target_scaler = StandardScaler()
T_train = target_scaler.fit_transform(T_train)
T_test = target_scaler.transform(T_test)

# ------------------------------
# Función auxiliar para crear el modelo
def build_model(activation):
    I = Input(shape=(X_train.shape[1],), name="entrada")
    if activation == "leakyrelu":
        h1 = Dense(64)(I)
        h1 = LeakyReLU(alpha=0.1)(h1)
        h2 = Dense(32)(h1)
        h2 = LeakyReLU(alpha=0.1)(h2)
    else:
        h1 = Dense(64, activation=activation)(I)
        h2 = Dense(16, activation=activation)(h1)
    out = Dense(2, activation='linear', name="salida")(h2)
    model = Model(inputs=I, outputs=out)
    model.compile(optimizer=Adam(learning_rate=ALPHA), loss=LOSS, metrics=['mse'])
    return model

# ------------------------------
# Entrenamiento repetido
results = {}
activations = ["tanh", "sigmoid", "relu", "leakyrelu"]

for act in activations:
    epochs_list = []
    ecm_list = []
    for r in range(REPS):
        model = build_model(act)
        early = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)
        history = model.fit(
            X_train, T_train,
            epochs=EPOCHS,
            batch_size=BATCH,
            validation_data=(X_test, T_test),
            verbose=1,
            callbacks=[early]
        )
        # épocas hasta detenerse
        epochs_list.append(len(history.history['loss']))
        # ECM en test
        ecm = model.evaluate(X_test, T_test, verbose=1)[0]
        ecm_list.append(ecm)
    results[act] = {
        "epocas_promedio": np.mean(epochs_list),
        "ecm_promedio": np.mean(ecm_list)
    }

# ------------------------------
# Mostrar resultados
print("\nResultados SGD (targets escalados):")
for act, vals in results.items():
    print(f"{act:10s} -> Épocas promedio: {vals['epocas_promedio']:.2f}, ECM promedio: {vals['ecm_promedio']:.4f}")

Epoch 1/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.6820 - mse: 0.6820 - val_loss: 0.4804 - val_mse: 0.4804
Epoch 2/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2784 - mse: 0.2784 - val_loss: 0.3314 - val_mse: 0.3314
Epoch 3/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.2069 - mse: 0.2069 - val_loss: 0.2377 - val_mse: 0.2377
Epoch 4/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1310 - mse: 0.1310 - val_loss: 0.1928 - val_mse: 0.1928
Epoch 5/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1037 - mse: 0.1037 - val_loss: 0.1775 - val_mse: 0.1775
Epoch 6/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0761 - mse: 0.0761 - val_loss: 0.1521 - val_mse: 0.1521
Epoch 7/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0652 - mse: 0.0652 - val_loss: 0.1530 - val_mse: 0.1530
Epoch 8/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0577 - mse: 0.0577 - val_loss: 0.1375 - val_mse: 0.1375
Epoch 9/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0507 - mse: 

/home/manuel/Documents/Facultad/DeepLearning/env/lib/python3.13/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.4885 - mse: 0.4885 - val_loss: 0.3226 - val_mse: 0.3226
Epoch 2/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1497 - mse: 0.1497 - val_loss: 0.2306 - val_mse: 0.2306
Epoch 3/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1149 - mse: 0.1149 - val_loss: 0.1632 - val_mse: 0.1632
Epoch 4/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0741 - mse: 0.0741 - val_loss: 0.1606 - val_mse: 0.1606
Epoch 5/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0652 - mse: 0.0652 - val_loss: 0.1347 - val_mse: 0.1347
Epoch 6/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0395 - mse: 0.0395 - val_loss: 0.1202 - val_mse: 0.1202
Epoch 7/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0322 - mse: 0.0322 - val_loss: 0.1143 - val_mse: 0.1143
Epoch 8/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0268 - mse: 0.0268 - val_loss: 0.1145 - val_mse: 0.1145
Epoch 9/700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0294 - mse: 0.0294 - val